In [1]:
import rclpy
from rclpy.node import Node
from sensor_msgs.msg import Image, PointCloud2
from cv_bridge import CvBridge
import cv2
import numpy as np
import pandas as pd
from time import time, sleep
from datetime import datetime

In [2]:
# Create directories for saving images
import os
left_images_dir = 'left_images'
right_images_dir = 'right_images'
point_clouds_dir = 'point_clouds'
os.makedirs(left_images_dir, exist_ok=True)
os.makedirs(right_images_dir, exist_ok=True)
os.makedirs(point_clouds_dir, exist_ok=True)

In [3]:
class SimpleDataCollector(Node):
    def __init__(self):
        super().__init__('simple_data_collector')
        self.bridge = CvBridge()

        self.last_timestamp = 0
        self.sync_rate = 1/100
        
        self.left_image = None
        self.right_image = None
        self.point_cloud = None

        self.left_time = 0
        self.right_time = 0
        self.point_cloud_time = 0

        self.left_image_sub = self.create_subscription(Image, '/cam_1/zed_node/left/image_rect_color', self.left_image_callback, 10)
        self.right_image_sub = self.create_subscription(Image, '/cam_1/zed_node/right/image_rect_color', self.right_image_callback, 10)
        self.point_cloud_sub = self.create_subscription(PointCloud2, '/cam_1/zed_node/point_cloud/cloud_registered', self.point_cloud_callback, 10)
        
    def synchronization(self):
        #if self.left_image is not None and self.right_image is not None and self.depth_image is not None:
        if self.left_time == self.right_time == self.point_cloud_time:

            cv_left = self.bridge.imgmsg_to_cv2(self.left_image, desired_encoding='bgr8')
            cv_right = self.bridge.imgmsg_to_cv2(self.right_image, desired_encoding='bgr8')
            npy_cloud = self.point_cloud2_to_npy(self.point_cloud)

            left_filename = f"{left_images_dir}/frame_{self.left_time}.jpg"
            right_filename = f"{right_images_dir}/frame_{self.right_time}.jpg"
            cloud_filename = f"{point_clouds_dir}/frame_{self.point_cloud_time}.csv"

            cv2.imwrite(left_filename, cv_left)
            cv2.imwrite(right_filename, cv_right)
            np.save(cloud_filename, npy_cloud)
            
            print('Current time is:' + str(self.left_time))
            interval = self.left_time - self.last_timestamp
            print('Time   interval:' + str(interval))

            self.last_timestamp = self.left_time
            
    def left_image_callback(self, msg):
        self.left_image = msg
        self.left_time = self.left_image.header.stamp.sec + self.left_image.header.stamp.nanosec / 1e9
        self.synchronization()

    def right_image_callback(self, msg):
        self.right_image = msg
        self.right_time = self.right_image.header.stamp.sec + self.right_image.header.stamp.nanosec / 1e9
        self.synchronization()

    def point_cloud_callback(self, msg):
        self.point_cloud = msg
        self.point_cloud_time = self.point_cloud.header.stamp.sec + self.point_cloud.header.stamp.nanosec / 1e9
        self.synchronization()
    
    def point_cloud2_to_npy(self, msg):
        from sensor_msgs_py import point_cloud2
        cloud_points = np.array(point_cloud2.read_points(msg, skip_nans=False))
        return cloud_points

In [4]:
rclpy.init()
simple_data_collector = SimpleDataCollector()
rclpy.spin(simple_data_collector)

Current time is:1707792798.1780202
Time   interval:1707792798.1780202
Current time is:1707792798.1946902
Time   interval:0.01666998863220215
Current time is:1707792798.2115293
Time   interval:0.016839027404785156
Current time is:1707792798.2615223
Time   interval:0.049993038177490234
Current time is:1707792798.2781854
Time   interval:0.016663074493408203
Current time is:1707792798.2948682
Time   interval:0.016682863235473633
Current time is:1707792798.3783243
Time   interval:0.08345603942871094
Current time is:1707792798.3950872
Time   interval:0.016762971878051758
Current time is:1707792798.4452062
Time   interval:0.05011892318725586
Current time is:1707792798.4618313
Time   interval:0.016625165939331055
Current time is:1707792798.4784992
Time   interval:0.016667842864990234
Current time is:1707792798.4952161
Time   interval:0.016716957092285156
Current time is:1707792798.5453281
Time   interval:0.050112009048461914
Current time is:1707792798.5618281
Time   interval:0.0164999961853027

KeyboardInterrupt: 